In [1]:
from fractal_zero.trainers.muzero_discriminator import FractalMuZeroDiscriminatorTrainer, FMZGModel
from fractal_zero.models.joint_model import JointModel

from fractal_zero.data.expert_dataset import ExpertDatasetGenerator
from fractal_zero.vectorized_environment import load_environment

import wandb
import torch
from tqdm import tqdm_notebook as tqdm

In [2]:
env = load_environment("CartPole-v0")

/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# get_expert_action = lambda x: env.action_space.sample()  # random policy

_expert_policy_model = torch.load("models/best_cartpole_policy.pth")
def get_expert_action(x):
    embedded_actions = _expert_policy_model.forward(x)
    return _expert_policy_model.parse_actions(embedded_actions)

# play with the expert for a bit
# obs = env.reset()
# total_reward = 0
# for _ in range(200):
#     action = get_expert_action(obs)
#     obs, reward, done, info = env.step(action)
#     total_reward += reward
#     env.render()
#     if done:
#         break

# env.close()
# print(f"total_reward={total_reward}")

In [4]:
# rep_model = torch.nn.Sequential(
#     torch.nn.Linear(4, 16),
#     torch.nn.ReLU(),
#     torch.nn.Linear(16, 16),
#     torch.nn.ReLU(),
#     torch.nn.Linear(16, 4),
#     torch.nn.ReLU(),
# )
rep_model = torch.nn.Identity()

dyn_model = torch.nn.Sequential(
    torch.nn.Linear(5, 16),  # obs space + action embedding = 4 + 1 = 5
    torch.nn.ReLU(),
    torch.nn.Sigmoid(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 4),
    torch.nn.Sigmoid(),  # give FMC a bit of an advantage (discrim tries to push embedding to higher magnitudes)
)

# the disc model receives the output embedding from the representation model
disc_model = torch.nn.Sequential(
    torch.nn.Linear(5, 1),
    torch.nn.Sigmoid(),  # between 0 and 1 confidence values.
)

action_vec = lambda x: torch.tensor(x).flatten().int().item()

model = FMZGModel(
    env,
    rep_model, 
    dyn_model, 
    disc_model, 
    num_walkers=256,
    action_vectorizer=action_vec,
)

expert_dataset = ExpertDatasetGenerator(
    get_expert_action,
    env,
    action_vectorizer=action_vec,
)

discriminator_optimizer = torch.optim.Adam([
    *rep_model.parameters(), 
    *dyn_model.parameters(),
    *disc_model.parameters(),
], lr=0.001)

trainer = FractalMuZeroDiscriminatorTrainer(env, model, expert_dataset, discriminator_optimizer)

In [5]:
# trainer._get_agent_trajectory(200, render=True)
# env.close()

In [6]:
wandb.init(project="fractal_muzero_cartpole")

/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: dyllan. Use `wandb login --relogin` to force relogin
/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:46: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


In [7]:
episodes = 100
train_steps = 10
max_steps = 200

for _ in tqdm(range(episodes), desc="Training Episodes"):
    trainer.generate_batches(max_steps)

    for _ in range(train_steps):
        loss = trainer.train_step()

/tmp/ipykernel_943303/3228566961.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm(range(episodes), desc="Training Episodes"):


Training Episodes:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_943303/2139027595.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action_vec = lambda x: torch.tensor(x).flatten().int().item()


In [ ]:
trainer.fmc.distances.mean()

In [ ]:
trainer._get_agent_trajectory(max_steps, render=True)

In [ ]:
len(trainer.fmc.tree.best_path)

In [ ]:
trainer.fmc.tree.best_path.total_reward

In [ ]:
env.close()

In [ ]:
trainer.fmc.tree.render()